day 1

In [ ]:
from queue import PriorityQueue

f = open('1_2.txt')
lines = f.read().split('\n')

max_sum, cur_sum = 0, 0
q = PriorityQueue()
for line in lines:
  if line != '':
    cur_sum += int(line)
  else:
    max_sum = max(max_sum, cur_sum)
    q.put(-cur_sum)
    cur_sum = 0

top_three_sum = 0
for i in range(3):
  top_three_sum -= q.get()

max_sum, top_three_sum

(69528, 206152)

day 2

In [ ]:
# given a list of elements,
# group into a list of lists of elements, delimited by `delimiter`
def group_by_delimiter(collection, delimiter=''):
  out = []
  group = []
  for e in collection:
    if e == delimiter:
      out.append(group)
      group = []
    else:
      group.append(e)
  return out

In [ ]:
shape_to_score = {
    'X': 1,
    'Y': 2,
    'Z': 3
}

opp_to_shape = {
    'A': 'X',
    'B': 'Y',
    'C': 'Z'
}

In [ ]:
def parse_rounds():
  s = '''
A Y
B X
C Z
  '''

  s = open('2_2.txt').read()
  rounds = []

  for line in s.split('\n')[:-1]:#[1:-1]:
    tokens = line.split(' ')
    opp, you = tokens[0], tokens[1]
    rounds.append([opp_to_shape[opp], you])
  return rounds

In [ ]:
def eval_total_score(rounds):
  # key => [draws with, beats, loses]
  lookup = {
      'X': ['X', 'Z', 'Y'],
      'Y': ['Y', 'X', 'Z'],
      'Z': ['Z', 'Y', 'X'],
  }

  total_score = 0
  for opp, you in rounds:
    draws_with, beats, loses = lookup[opp]
    if draws_with == you: # opp and you draw with each other
      total_score += shape_to_score[you] + 3
    elif beats == you: # opp beats, you lose
      total_score += shape_to_score[you]
    elif loses == you: # opp loses, you beat
      total_score += shape_to_score[you] + 6
  return total_score

eval_total_score(parse_rounds())

14163

In [ ]:
result_to_score = {
    'X': 0, # you lose
    'Y': 3, # draw
    'Z': 6, # you win
}

def get_move_for_result(opp, result):
  lookup = {
    'X': ['X', 'Z', 'Y'],
    'Y': ['Y', 'X', 'Z'],
    'Z': ['Z', 'Y', 'X'],
  }
  options = lookup[opp]
  if result == 'Y': # draw
    return options[0]
  elif result == 'X': # you lose
    return options[1]
  elif result =='Z': # you win
    return options[2]

def eval_total_score_2(rounds):
  total_score = 0
  for opp, result in rounds:
    you = get_move_for_result(opp, result)
    total_score += shape_to_score[you] + result_to_score[result]
  return total_score

eval_total_score_2(parse_rounds())

12091

day 3

In [ ]:
def parse(fn):
  sacks = []
  with open(fn) as fd:
    for line in fd.readlines():
      line = line.strip()
      length = len(line) // 2
      left, right = line[:length], line[length:]
      sacks.append([left, right])
  return sacks

In [ ]:
def priority(char):
  if 'a' <= char and char <= 'z':
    return ord(char) - 96
  if 'A' <= char and char <= 'Z':
    return ord(char) - ord('A') + 27

def p1(sacks):
  priorities = 0
  for (left, right) in sacks:
    common_char = set(left).intersection(set(right))
    common_char = next(iter(common_char))
    priorities += priority(common_char)
  return priorities

p1(parse('3_2.txt'))

7863

In [ ]:
def p2(sacks):
  ctr = 0
  groups, group = [], []
  for i, (left, right) in enumerate(sacks):
    group.append(left + right)
    ctr += 1
    if ctr == 3:
      groups.append(group)
      ctr, group = 0, []

  priorities = 0
  for group in groups:
    common_char = set(group[0]).intersection(set(group[1])).intersection(set(group[2]))
    common_char = next(iter(common_char))
    priorities += priority(common_char)
  return priorities

p2(parse('3_2.txt'))

2488

day 4

In [ ]:
def parse(fn):
  range_pairs = []
  with open(fn) as fd:
    lines = fd.read().split('\n')[:-1]
    print(lines)
    for line in lines:
      p1, p2 = line.split(',')
      p1_1, p1_2 = list(map(lambda x: int(x), p1.split('-')))
      p2_1, p2_2 = list(map(lambda x: int(x), p2.split('-')))
      range_pairs.append(((p1_1, p1_2), (p2_1, p2_2)))
  return range_pairs

In [ ]:
def overlaps(p1_1, p1_2, p2_1, p2_2):
  #return (p2_1 >= p1_1 and p2_2 <= p1_2) or (p1_1 >= p2_1 and p1_2 <= p2_2)
  return max(p1_1, p2_1) <= min(p1_2, p2_2)

def p1(range_pairs):
  out = 0
  for ((p1_1, p1_2), (p2_1, p2_2)) in range_pairs:
    if overlaps(p1_1, p1_2, p2_1, p2_2):
      out += 1
  return out

p1(parse('4_2.txt'))

['2-88,13-89', '12-94,12-94', '34-69,34-61', '9-76,9-9', '35-82,34-52', '9-10,10-88', '57-71,71-75', '50-71,71-91', '44-67,43-43', '68-69,67-68', '6-47,6-48', '6-52,7-67', '43-51,40-44', '56-67,57-69', '94-94,68-94', '10-97,9-9', '40-95,40-98', '22-81,22-22', '7-58,7-59', '8-82,82-82', '22-27,23-44', '5-27,7-28', '11-75,11-75', '7-90,99-99', '65-89,65-66', '24-52,29-53', '41-57,27-58', '14-95,13-26', '27-34,26-75', '18-33,33-56', '21-95,21-95', '20-29,69-71', '81-81,23-90', '47-83,47-82', '28-50,86-98', '50-76,21-50', '3-69,64-79', '17-97,10-16', '54-92,10-93', '7-75,45-77', '72-72,42-73', '13-80,81-87', '25-26,25-64', '38-48,38-38', '2-99,2-99', '48-93,67-92', '12-85,2-94', '14-75,7-82', '29-46,23-29', '50-94,4-51', '3-37,37-91', '24-91,91-91', '25-48,44-47', '2-77,1-26', '1-58,1-21', '16-93,16-93', '3-99,8-98', '23-23,6-23', '17-67,68-68', '4-15,1-4', '35-41,35-40', '6-88,52-89', '11-48,11-61', '4-64,12-65', '7-90,7-8', '1-89,6-73', '64-69,38-70', '26-67,59-66', '41-54,16-42', '2-99,

883

day 5

In [ ]:
import os
import re

def parse(fn):
  BLANK_LINE = os.linesep + os.linesep
  stacks_str, moves_str = open(fn).read().split(BLANK_LINE)
  n_stacks = int(stacks_str[-2]) # assumes n_stacks <= 9
  rows, moves = [], []

  for stack_row_str in stacks_str.split('\n')[:-1]:
    rows.append(stack_row_str
      .replace('] [', ',').replace('[', '').replace(']', '')
      .replace(' '*4, ',')
      .replace(' ', ',')
      .split(',')
    )

  for move_str in moves_str.split('\n')[:-1]:
    move = re.findall('move ([0-9]+) from ([0-9]+) to ([0-9]+)', move_str)
    moves.append([int(x) for x in move[0]])

  stacks = list([
    list(filter(lambda crate: crate != '', col)) for col in zip(*rows)
  ])

  return stacks, moves

In [ ]:
def p1(stacks, moves):
  for (n_to_move, from_idx, to_idx) in moves:
    for _ in range(n_to_move):
      stacks[to_idx - 1] = [stacks[from_idx - 1].pop(0)] + stacks[to_idx - 1]
  return ''.join([stack[0] for stack in stacks])

stacks, moves = parse('5_2.txt')
print(p1(stacks, moves))

RFFFWBPNS


In [ ]:
def p1(stacks, moves):
  for (n_to_move, from_idx, to_idx) in moves:
    stacks[to_idx - 1] = stacks[from_idx - 1][:n_to_move] + stacks[to_idx - 1]
    stacks[from_idx - 1] = stacks[from_idx - 1][n_to_move:]
  return ''.join([stack[0] for stack in stacks])

stacks, moves = parse('5_2.txt')
print(p1(stacks, moves))

CQQBBJFCS


day 6

In [ ]:
def parse(fn):
  return open(fn).read().strip()

def p1(s):
  substr = list(s[:4])
  for i, c in enumerate(s[4:]):
    substr_set = set(substr)

    if len(substr_set) == len(substr):
      return i + 4

    delete_c = substr.pop(0)
    substr.append(c)

  if len(substr_set) == len(substr):
    return i + 1
  return ''

p1('bvwbjplbgvbhsrlpgdmjqwftvncz')

5

In [ ]:
p1('nznrnfrfntjfmvfwmzdfjlvtqnbhcprsg')

10

In [ ]:
p1('mjqjpqmgbljsphdztnvjfqwrcgsmlb')

7

In [ ]:
p1('zcfzfwzzqfrljwzlrfnpqdbhtmscgvjw')

11

In [ ]:
p1(parse('6_2.txt'))

1262

In [ ]:
def p2(s):
  substr = list(s[:14])
  for i, c in enumerate(s[14:]):
    substr_set = set(substr)
    if len(substr_set) == len(substr):
      return i + 14
    substr.pop(0)
    substr.append(c)

  return ''

p2(parse('6_2.txt'))

3444